# Day 5: Binary Boarding

https://adventofcode.com/2020/day/5

## Part 1

In [2]:
from pathlib import Path
from typing import Tuple

INPUTS = Path('input.txt').resolve().read_text().strip()


class CodedSeat:

    ROW_LOWER_CHAR = "F"
    ROW_UPPER_CHAR = "B"
    COL_LOWER_CHAR = "L"
    COL_UPPER_CHAR = "R"

    def __init__(self, code):
        self.code = code
    
    def __str__(self):
        return self.code
    
    @property
    def row_str(self):
        return self.code[:-3]
    
    @property
    def col_str(self):
        return self.code[-3:]
    
    @staticmethod
    def get_binary_num(value: str, lower_char:str, upper_char:str) -> int:
        """Convert a coded string to its number equivalent."""
        value = value.upper()
        if value.replace(lower_char, '').replace(upper_char, ''):
            # Invalid characters in this string
            raise ValueError("Invalid characters found, should only be %s and %s!" % (lower_char, upper_char))
        lower_bound = 0
        upper_bound = (2 ** len(value)) - 1  # 127 for 7 characters
        for char in value:
            diff = (upper_bound + 1 - lower_bound) // 2
            if char == lower_char:
                # Lower half: decrease the upper bound
                upper_bound -= diff
            else:
                # Upper half: increase the lower bound
                lower_bound += diff
        # The two bounds at this point should be identical, so return one of them
        return lower_bound
    
    @property
    def row_num(self):
        return self.get_binary_num(self.row_str, self.ROW_LOWER_CHAR, self.ROW_UPPER_CHAR)
    
    @property
    def col_num(self):
        return self.get_binary_num(self.col_str, self.COL_LOWER_CHAR, self.COL_UPPER_CHAR)
    
    @property
    def seat_id(self):
        return (self.row_num * 8) + self.col_num

# Our sanity check asks us to get the highest seat ID of the bunch.
# According to the algorithm, 'B' amounts to a higher-order row,
# while 'F' amounts to a lower-order row.
# That means if we simply perform a sort on the list of seats,
# naturally the first few will be in the highest rows in the plane.

# It will not naturally sort the columns, as 'R' is the upper half
# while 'L' is the lower. In fact, that will give us the lowest ID
# within the highest row of the plane, alongside others that are also
# in that row.

# So, first, let's find the identity of that highest row,
# cutting off the col numbers:

seats = INPUTS.split()

highest_row = sorted(seats)[0][:-3]
print(f"Highest row string: {highest_row}")

# We can then pull out all the seats that start with this row identifier:

highest_row_seats = [x for x in seats if x.startswith(highest_row)]

# Then if we reverse-sort this list and take the first one, we have our highest seat:

highest_seat = CodedSeat(sorted(highest_row_seats, reverse=True)[0])
print(f"Highest seat in that row: {highest_seat}")
print(f"  ID of that seat: {highest_seat.seat_id}")

# I constructed the CodedSeat class after passing the sanity check using pure functions,
# then refactored to include it.
# With the class in hand, another way to solve this is simply to generate IDs for every seat,
# then sort and pull out the highest one:

all_seats = [CodedSeat(x) for x in seats]
highest_seat_id = sorted([x.seat_id for x in all_seats], reverse=True)[0]
print(highest_seat_id)


Highest row string: BBFBBFF
Highest seat in that row: BBFBBFFRRR
  ID of that seat: 871
871


## Part 2

In [4]:
# Using the highest seat ID on the plane, we know an upper limit for all the seats,
# so building a mapping of all possible seats is trivial.
mapped_seats = {x: False for x in range(highest_seat_id + 1)}

# We can then map all seats from our input to its ID key in mapped_seats.
# We just need to know the seat is filled, so we'll flip the flag in that
# location to True.
for seat in all_seats:
    mapped_seats[seat.seat_id] = True

# With all filled seats mapped out, we can filter to find the remaining missing ones:
missing_seats = [k for k, v in mapped_seats.items() if v is False]

# Now, according to the rules, our seat is not necessarily in the first seat that equates to False.
# We must find a missing seat `n` (one of those identified in `missing_seats`) where
# both `n+1` and `n-1` in `mapped_seats` are True.

for missing_id in missing_seats:
    if (missing_id - 1) not in mapped_seats or (missing_id + 1) not in mapped_seats:
        # There are no seats surrounding this one,
        # so it's at the top or bottom of the plane,
        # and thus not our real seat
        continue
    if not mapped_seats[missing_id-1] or not mapped_seats[missing_id+1]:
        # The nearby seats are also "empty", meaning they probably don't exist.
        continue
    # passing those two conditions, the missing seat ID we found is the only one remaining.
    our_seat = missing_id
    # We might missing seats at the back of the plane, as well, but we can be sure that
    # those seats will fail the prior conditions. Therefore, we can exit early.
    break

# And here's our result.
print(f"Our seat is #{our_seat}.")

Our seat is #640.
